In [1]:
import os
from dotenv import load_dotenv, find_dotenv

In [2]:
load_dotenv(find_dotenv())
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

## Document Loader

In [4]:
loader = TextLoader('input/Sample.txt')
documents = loader.load()

In [5]:
len(documents)

1

## Document transformers

In [6]:
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

Created a chunk of size 486, which is longer than the specified 200
Created a chunk of size 426, which is longer than the specified 200
Created a chunk of size 419, which is longer than the specified 200
Created a chunk of size 498, which is longer than the specified 200
Created a chunk of size 319, which is longer than the specified 200
Created a chunk of size 436, which is longer than the specified 200
Created a chunk of size 309, which is longer than the specified 200
Created a chunk of size 365, which is longer than the specified 200
Created a chunk of size 288, which is longer than the specified 200
Created a chunk of size 320, which is longer than the specified 200
Created a chunk of size 354, which is longer than the specified 200
Created a chunk of size 263, which is longer than the specified 200
Created a chunk of size 339, which is longer than the specified 200
Created a chunk of size 338, which is longer than the specified 200
Created a chunk of size 366, which is longer tha

In [7]:
len(texts)

19

In [8]:
texts

[Document(page_content='India, officially known as the Republic of India, is a diverse and vibrant country located in South Asia. With a rich history spanning thousands of years, India is known for its cultural heritage, religious diversity, and vast landscapes. From the majestic Himalayas in the north to the serene backwaters of Kerala in the south, India encompasses a wide range of geographical features, including deserts, plains, mountains, and coastlines, making it a land of incredible natural beauty.', metadata={'source': 'input/Sample.txt'}),
 Document(page_content='India is the seventh-largest country by land area and the second-most populous country in the world, with a population exceeding 1.3 billion people. It is a federal parliamentary democratic republic, with a president as the head of state and a prime minister as the head of government. The country follows a multi-tiered administrative structure, with 28 states and 9 union territories, each having its own elected govern

## Text embeddinf models

In [9]:
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

/Users/gota/jupyter-lab/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Vector stores

In [10]:
db = Chroma.from_documents(texts, embeddings)

In [11]:
db._collection.get(include=["embeddings"])

{'ids': ['c4318552-3a12-11ee-9f17-762193880224',
  'c43185ca-3a12-11ee-9f17-762193880224',
  'c43185de-3a12-11ee-9f17-762193880224',
  'c43185f2-3a12-11ee-9f17-762193880224',
  'c43185fc-3a12-11ee-9f17-762193880224',
  'c4318610-3a12-11ee-9f17-762193880224',
  'c431861a-3a12-11ee-9f17-762193880224',
  'c4318638-3a12-11ee-9f17-762193880224',
  'c431864c-3a12-11ee-9f17-762193880224',
  'c4318660-3a12-11ee-9f17-762193880224',
  'c4318674-3a12-11ee-9f17-762193880224',
  'c431867e-3a12-11ee-9f17-762193880224',
  'c4318688-3a12-11ee-9f17-762193880224',
  'c4318692-3a12-11ee-9f17-762193880224',
  'c43186a6-3a12-11ee-9f17-762193880224',
  'c43186b0-3a12-11ee-9f17-762193880224',
  'c43186ba-3a12-11ee-9f17-762193880224',
  'c43186ce-3a12-11ee-9f17-762193880224',
  'c43186d8-3a12-11ee-9f17-762193880224'],
 'embeddings': [[0.0860704556107521,
   0.021627152338624,
   -0.04143557697534561,
   0.02944786846637726,
   0.013506031595170498,
   -0.02148868516087532,
   0.046076852828264236,
   -0.01112

## Retrievers

In [12]:
retriever = db.as_retriever(search_kwargs={"k": 1})

In [13]:
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], metadata=None, vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x1776ec490>, search_type='similarity', search_kwargs={'k': 1})

## QA

In [15]:
docs = retriever.get_relevant_documents("what is the capital of India?")

In [16]:
docs

[Document(page_content='Delhi is the capital of India', metadata={'source': 'input/Sample.txt'})]